# Jaeger-query with trace DSL

This notebook loads a trace from Jaeger query service and runs analysis on it.

## Install library to the local maven repository
This step is only needed if trace DSL source code has been modified.
Open terminal in Jupyter and run the following command:
```
cd work && ./mvnw clean install -DskipTests
```

In [1]:
%%loadFromPOM
<dependency>
    <groupId>com.fasterxml.jackson.core</groupId>
    <artifactId>jackson-databind</artifactId>
    <version>2.6.7</version>
</dependency>
<dependency>
    <groupId>com.fasterxml.jackson.core</groupId>
    <artifactId>jackson-annotations</artifactId>
    <version>2.6.7</version>
</dependency>
<dependency>
    <groupId>io.prometheus</groupId>
    <artifactId>simpleclient_httpserver</artifactId>
    <version>0.7.0</version>
</dependency>
<dependency>
    <groupId>org.apache.tinkerpop</groupId>
    <artifactId>tinkergraph-gremlin</artifactId>
    <version>3.4.3</version>
</dependency>
<dependency>
    <groupId>log4j</groupId>
    <artifactId>log4j</artifactId>
    <version>1.2.17</version>
</dependency>


In [2]:
%maven io.jaegertracing:jaeger-tracedsl:1.0-SNAPSHOT

System.out.println(Keys.class);

class Keys


## Define Jaeger query URL and trace ID

In [4]:
String queryUrl = "http://192.168.122.1:16686/api/traces/";
String traceId = "3af34324f870e57e";

## Run analysis
It will display maximum height of the trace

In [5]:
import org.apache.tinkerpop.gremlin.structure.Graph;
import org.apache.tinkerpop.gremlin.structure.Vertex;
import io.jaegertracing.analytics.gremlin.*;
import io.jaegertracing.analytics.query.*;
import io.jaegertracing.analytics.model.*;

Trace trace = JaegerQueryService.load(traceId, queryUrl);
Graph graph = GraphCreator.create(trace);
int height = TraceHeight.calculate(graph);
System.out.println(height);


3


## Write your own data analysis with trace DSL

In [10]:
TraceTraversal<Vertex, Vertex> traversal = graph.traversal(TraceTraversalSource.class)
        .hasName("/api/traces/{traceID}")
        .repeat(__.out())
        .until(__.hasName("multiRead"));

traversal.forEachRemaining(v -> GraphDSLExamples.printVertex(v));


multiRead[3af34324f870e57e:53e1f3ef200f8a1], tags = [traceId, spanId, duration, internal.span.format, startTime, operationName, parentId]
